In [8]:
#CONVERT PDF TO PLAINTEXT FILES https://euske.github.io/pdfminer/programming.html
import pdfminer
from six import StringIO
import re
import sys
import os
sys.path.append("/usr/local/lib/python3.7/site-packages/pdfminer/")

from pdfminer.pdfparser import PDFParser
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfpage import PDFTextExtractionNotAllowed
from pdfminer.pdfinterp import PDFResourceManager
from pdfminer.pdfinterp import PDFPageInterpreter
from pdfminer.pdfdevice import PDFDevice
from pdfminer.layout import LAParams
from pdfminer.converter import PDFPageAggregator
from pdfminer.layout import LAParams, LTTextBox, LTTextLine

#CHANGE THESE PATHS TO THE FOLDER WHERE YOU STORE THE DATA
Path = "/Users/phuong/Documents/JBI_paper/Fulltext_Papers_PDFs/"
Output_Path = "/Users/phuong/Documents/JBI_paper/Fulltext_Papers_Plaintexts/"

#read the PDF files to get plain text full-text
filelist = os.listdir(Path)
for i in filelist:
    if i.endswith(".pdf"):
        # Open a PDF file.
        fp = open(Path + i, 'rb')
        # Create a PDF parser object associated with the file object.
        parser = PDFParser(fp)
        # Create a PDF document object that stores the document structure.
        # Supply the password for initialization.
        document = PDFDocument(parser)
        # Check if the document allows text extraction. If not, abort.
        if not document.is_extractable:
            raise PDFTextExtractionNotAllowed
        # Create a PDF resource manager object that stores shared resources.
        rsrcmgr = PDFResourceManager()
        # Set parameters for analysis.
        laparams = LAParams()
        # Create a PDF page aggregator object.
        device = PDFPageAggregator(rsrcmgr, laparams=laparams)
        interpreter = PDFPageInterpreter(rsrcmgr, device)
        extracted_text = ''
        for page in PDFPage.create_pages(document):
            interpreter.process_page(page)
            layout = device.get_result()
            for lt_obj in layout:
                if isinstance(lt_obj, LTTextBox) or isinstance(lt_obj, LTTextLine):
                    extracted_text = extracted_text + " "
                    extracted_text += lt_obj.get_text()

        outputFile = Output_Path + i.replace("pdf","txt")

        with open(outputFile,"wb") as txt_file:
            txt_file.write(extracted_text.encode("utf-8"))
            
            

In [11]:
#GET THE METHODS SECTIONS TO PLAINTEXT FILES
from io import BytesIO as StringIO
import re
import sys
import os

#CHANGE THESE PATHS TO THE FOLDER WHERE YOU STORE THE FULL-TEXT PLAINTEXT DATA
Output_Path = "/Users/phuong/Documents/JBI_paper/Fulltext_Papers_Plaintexts/"
#Get the text between Methods & Results section 
filelist_2 = os.listdir(Output_Path)
start_pattern = [" METHODS"," Methods"," Subjects and methods"," Subjects and Methods",
                 " MATERIALS AND METHODS"," Materials and methods",
                 " METHODS AND MATERIALS"," PATIENTS  AND  METHODS",
                 " PARTICIPANTS AND METHODS",
                 " SUBJECTS AND METHODS",
                 " Materials and Methods"]
stop_pattern = [" RESULTS"," Results"," RESULTS AND DISCUSSION"]
for i in filelist_2:
    if i.endswith(".txt"):
        with open(Output_Path + i, 'r') as article_plaintext_file:
            file_path = Output_Path + i
            text_between_Methods_Results = []
            recording = False
            output_section = []
            
            for line in article_plaintext_file:
                line = line.replace("\n", "")
                if recording is False:
                    #If the line contains "METHODS" header, start record the text, change the flag to "TRUE"
                    if line in start_pattern:
                        recording = True
                        output_section.append(line.strip())
                elif recording is True:
                    #If the line contains "RESULTS" header, stop record the text, change the flag to "FALSE"
                    if line in stop_pattern:
                        recording = False
                    else:
                        output_section.append(line.strip())    
                        
            #Save the methods section into a text file
            #CHANGE THESE PATHS TO THE FOLDER WHERE YOU STORE THE DATA
            methods_section_path = '/Users/phuong/Documents/JBI_paper/MethodsSections_Papers_Plaintexts/'
            methods_section_file_name = methods_section_path + "methods_section_" + i
            methods_section_file = open(methods_section_file_name,'w')  
            for line in output_section:
                line = line.strip()
                line = line.replace("\n", " ")
                line = line + " "
                methods_section_file.write(line)           
            methods_section_file.close()
    

In [12]:
#PREPROCESS TEXT OF THE METHODS SECTION

#remove stopword in "Methods section" text files:         
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
#word_tokenize accepts a string as an input, not a file. 
stop_words = set(stopwords.words('english')) 

#CHANGE THESE PATHS TO THE FOLDER WHERE YOU STORE THE DATA
Methods_Section_Path = "/Users/phuong/Documents/JBI_paper/MethodsSections_Papers_Plaintexts/"
filelist_3 = os.listdir(Methods_Section_Path)
for i in filelist_3:
    if i.endswith(".txt") and i.startswith("methods"):
        with open(Methods_Section_Path + i, 'r') as file1:
            #Preprocess the text from Methods section: eliminate stop words, eliminate numbers.
            line1 = file1.read()# Use this to read file content as a stream
            line1 = line1.replace("- ","")
            line1 = re.sub(' +', ' ',line1)
            words = line1.split() 
            methods_section_file_name_filtered = Methods_Section_Path + "methods_section_filtered_" + i
            for word in words:
                word = re.sub(r'\W+', ' ', word)
                word = word.lstrip()
                word = word.rstrip()
                if not word in stop_words: 
                    if not re.match("^[0-9 ]+$", word):
                        appendFile = open(methods_section_file_name_filtered,'a') 
                        appendFile.write(" "+ word) 
                        appendFile.close()


In [14]:
#remove stopword in "Methods section" text files:    
import nltk
import re
nltk.download('stopwords')
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
#word_tokenize accepts a string as an input, not a file. 
stop_words = set(stopwords.words('english')) 

import pandas as pd
newdata = pd.read_csv("/Users/phuong/Downloads/NewData_Metadata.csv",encoding='latin-1')

methods_section = newdata["Methods_Section"]
filtered_methods_section = []

for methods in methods_section:
    filtered_methods = []
    methods = methods.replace("- ","")
    methods = re.sub(' +', ' ',methods)
    words = methods.split() 
    for word in words:
        word = re.sub(r'\W+', ' ', word)
        word = word.lstrip()
        word = word.rstrip()
        if not word in stop_words: 
            if not re.match("^[0-9 ]+$", word):
                filtered_methods.append(word)
    filtered_methods = ' '.join(filtered_methods)
    print (filtered_methods)
    filtered_methods_section.append(filtered_methods)

newdata['Filtered_Methods_Section'] = filtered_methods_section 

newdata.to_csv("/Users/phuong/Downloads/NewData_Metadata_Filtered.csv")
    

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/phuong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Methods Study quality standards After approval responsible Ethics Committee Federal Institute Drugs Medical Devices BfArM Bonn Germany study EudraCT conducted Clinical Research Center Department Clinical Pharmacology Pharmacoepidemiology University Heidelberg certified according DIN EN ISO9001 2008 It followed standards Good Clinical Practice Declaration Helsinki specific legal requirements Germany Written informed consent obtained participant inclusion Population The participants screened presence functional SLCO1B1 15 haplotype CYP2C9 2 alleles Four groups healthy volunteers enrolled classified follows carriers wild type alleles CYP2C9 extensive metabolizers SLCO1B1 EM wt ii carriers wild type CYP2C9 deficient SLCO1B1 EM def iii poor metabolizers CYP2C9 wild type SLCO1B1 PM wt iv deficient PM def None participants regular drug treatment past months study except oral contraceptives females none taking drugs substances known induce inhibit drug metabolizing enzymes transporters within 

Methods Subjects A total healthy White volunteers men women participated study giving written informed consent The participants recruited pool subjects genotyped SLCO1B1 CYP2C8 CYP2C9 SNPs All genotyping performed TaqMan allelic discrimination Applied Biosystems Real Time Polymerase Chain Reaction System Applied Biosystems Foster City CA USA according manufacturer s instructions reaction volume µl Genotyping SLCO1B1 SNPs carried described previously Genotyping CYP2C9 2 c 430C_T CYP2C9 3 c 1075A_C alleles performed using TaqMan Pre Developed Assay Reagents Allelic Discrimination Applied Biosystems Genotyping CYP2C8 3 allele c 416G_A c 1196A_G carried using Custom TaqMan SNP genotyping assays Applied Biosystems CYP2C8 genotyping validated previously described method Only noncarriers CYP2C8 3 CYP2C9 2 CYP2C9 3 alleles recruited The participants selected basis SLCO1B1 c 521T_C SNP well g 11187G_A g 10499A_C c 388A_G SNPs allocated one three groups according genotype Haplotypes assigned des